### Student Name: Roshnaey Khattak

In [1]:
#TODO: import the modules
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np

### Step 1 – Load the data
Load `trip_data.csv`, select the salient features and engineer new features (if necessary).

In [2]:
#TODO: Load the data
file_path   = '/Users/roshnaey/Desktop/trip_data.csv'
df = pd.read_csv(file_path)
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,tip
0,2.0,2019-12-01 00:26:05,2019-12-01 00:31:30,1.0,24,41,1.0,0.67,5.5,0.5,0.5,0.0,0.3,8.84,1.0,0.00,good
1,2.0,2019-12-01 00:56:08,2019-12-01 01:05:11,1.0,112,232,1.0,2.64,10.0,0.5,0.5,0.0,0.3,16.86,1.0,2.75,low
2,2.0,2019-12-01 00:10:32,2019-12-01 00:34:48,1.0,97,227,5.0,3.85,18.0,0.5,0.5,0.0,0.3,22.20,1.0,0.00,low
3,2.0,2019-12-01 00:32:08,2019-12-01 00:44:06,1.0,223,129,1.0,2.97,11.5,0.5,0.5,0.0,0.3,13.80,1.0,0.00,low
4,2.0,2019-12-01 00:13:31,2019-12-01 00:22:38,1.0,223,7,1.0,1.60,8.0,0.5,0.5,0.0,0.3,11.62,1.0,0.00,good


In [3]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

In [4]:
pickup = df.lpep_pickup_datetime
dropoff = df.lpep_dropoff_datetime
elapsed_time = dropoff - pickup

In [5]:
trip_duration = pd.to_numeric(elapsed_time.dt.seconds, downcast = 'integer')

In [6]:
df['trip_duration'] = trip_duration
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,tip,trip_duration
0,2.0,2019-12-01 00:26:05,2019-12-01 00:31:30,1.0,24,41,1.0,0.67,5.5,0.5,0.5,0.0,0.3,8.84,1.0,0.00,good,325
1,2.0,2019-12-01 00:56:08,2019-12-01 01:05:11,1.0,112,232,1.0,2.64,10.0,0.5,0.5,0.0,0.3,16.86,1.0,2.75,low,543
2,2.0,2019-12-01 00:10:32,2019-12-01 00:34:48,1.0,97,227,5.0,3.85,18.0,0.5,0.5,0.0,0.3,22.20,1.0,0.00,low,1456
3,2.0,2019-12-01 00:32:08,2019-12-01 00:44:06,1.0,223,129,1.0,2.97,11.5,0.5,0.5,0.0,0.3,13.80,1.0,0.00,low,718
4,2.0,2019-12-01 00:13:31,2019-12-01 00:22:38,1.0,223,7,1.0,1.60,8.0,0.5,0.5,0.0,0.3,11.62,1.0,0.00,good,547


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['tip'].value_counts()
le.fit(df.tip)
le.classes_
list(le.classes_)
le.transform(df.tip)
list(le.inverse_transform([2, 1, 3, 0]))
df['tip_labeled'] = le.transform(df.tip)

In [8]:
del df['tip']
del df['lpep_pickup_datetime']
del df['lpep_dropoff_datetime']
df.head()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,trip_duration,tip_labeled
0,2.0,1.0,24,41,1.0,0.67,5.5,0.5,0.5,0.0,0.3,8.84,1.0,0.00,325,0
1,2.0,1.0,112,232,1.0,2.64,10.0,0.5,0.5,0.0,0.3,16.86,1.0,2.75,543,1
2,2.0,1.0,97,227,5.0,3.85,18.0,0.5,0.5,0.0,0.3,22.20,1.0,0.00,1456,1
3,2.0,1.0,223,129,1.0,2.97,11.5,0.5,0.5,0.0,0.3,13.80,1.0,0.00,718,1
4,2.0,1.0,223,7,1.0,1.60,8.0,0.5,0.5,0.0,0.3,11.62,1.0,0.00,547,0


### Step 2 – Split the data

In [9]:
from sklearn.model_selection import train_test_split 

#TODO: Split the data into training and test sets
X = df.iloc[:,0:15].to_numpy() #input features
y = df.iloc[:,15].tolist() #labels

seed = 1234

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.4 , #TODO: provide the test_size (refer to the instructions for the assignment)
                                                    random_state = seed, #seed the random generator for reproducibilty
                                                    stratify = y
                                                   )

### Step 3 – Create the Classifier
Use the [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) in scikit-learn and optimize the parameters to achieve a prediction accuracy of 95% (or higher).

In [10]:
#Create the RandomForestClassifier
rf_classifier = RandomForestClassifier(criterion="entropy" ,  #TODO: provide an option 
                                    max_depth= 15,     #TODO: provide a value. The depth should not exceed 15.
                                    max_features= 10 ,  #TODO: provide a suitable value
                                    min_samples_split= 2, #TODO: provide a suitable value
                                    n_estimators= 200, #TODO: provide a suitable value
                                    random_state= 7 #TODO: provide a suitable value
                                    )


In [11]:
#TODO: Train classifer using the `fit` method
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=15, max_features=10,
                       n_estimators=200, random_state=7)

In [12]:
#TODO: Make predictions with the test set
y_prediction = rf_classifier.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score
#TODO: evaluate the accuracy
accuracy_score(y_test, y_prediction)

0.9509138229349928

### Step 4 – Make predictions with new data

In [14]:
#TODO: Use the model to make predictions with the supplied test set: `new_trips.csv`
new_file_path  = '/Users/roshnaey/Desktop/module07-random-forest-roshnaey-main/new_trips.csv'
new_df = pd.read_csv(new_file_path)
new_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge
0,1,2019-02-14T05:45:41.000,2019-02-14T06:04:01.000,1,42,230,1,5.40,18.00,3.25,0.5,0.00,0.3,24.05,1,2.75
1,2,2019-02-14T05:46:15.000,2019-02-14T05:56:21.000,1,75,137,3,4.88,15.00,0.50,0.5,0.00,0.3,19.05,1,2.75
2,2,2019-02-14T05:46:47.000,2019-02-14T06:13:06.000,5,254,74,1,9.71,28.33,0.00,0.5,0.00,0.0,28.83,2,0.00
3,2,2019-02-14T05:47:11.000,2019-02-14T06:05:15.000,5,29,39,1,8.86,25.38,0.00,0.5,0.00,0.0,25.88,2,0.00
4,2,2019-02-14T05:47:38.000,2019-02-14T06:03:45.000,1,75,138,1,8.09,23.50,0.50,0.5,5.76,0.3,36.67,1,0.00


In [15]:
new_df['lpep_pickup_datetime'] = pd.to_datetime(new_df['lpep_pickup_datetime'])
new_df['lpep_dropoff_datetime'] = pd.to_datetime(new_df['lpep_dropoff_datetime'])
pickup2 = new_df.lpep_pickup_datetime
dropoff2 = new_df.lpep_dropoff_datetime
elapsed_time2 = dropoff2 - pickup2
trip_duration2 = pd.to_numeric(elapsed_time.dt.seconds, downcast = 'integer')
new_df['trip_duration'] = trip_duration2

In [16]:
del new_df['lpep_pickup_datetime']
del new_df['lpep_dropoff_datetime']
new_df.head()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,trip_duration
0,1,1,42,230,1,5.40,18.00,3.25,0.5,0.00,0.3,24.05,1,2.75,325
1,2,1,75,137,3,4.88,15.00,0.50,0.5,0.00,0.3,19.05,1,2.75,543
2,2,5,254,74,1,9.71,28.33,0.00,0.5,0.00,0.0,28.83,2,0.00,1456
3,2,5,29,39,1,8.86,25.38,0.00,0.5,0.00,0.0,25.88,2,0.00,718
4,2,1,75,138,1,8.09,23.50,0.50,0.5,5.76,0.3,36.67,1,0.00,547


In [17]:
new_data_prediction = rf_classifier.predict(new_df)

In [18]:
print (new_data_prediction)

[1 2 2 2 3 2 3 3 1 3 3 3 3 3 1 3 3 1 3 1]


### Step 5 – Summary
Summarize the results and explain the reason that ensemble methods can improve a model's ability to generalize.

In [ ]:
#TODO: Explain the results
## Ensemble models, such as random forest models, do not follow the same tree model as classic decision trees. 
## Instead of following a single feature like decision trees, ensemble models take into account multiple features 
## of a dataset to come to a conclusion. The algorithm takes into account the results of the decision tree from 
## each of the features, and makes a decision based on the weights of each feature. Since ensemble methods use 
## multiple decision trees to create models, they allow for the overall model to be more generalized because the 
## influence of each feature are generally the same. 